In [2]:
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import gc

2024-10-10 18:54:19.842929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 18:54:19.871643: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 18:54:19.879490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 18:54:19.897762: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpus=tf.config.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)


I0000 00:00:1728586465.966692     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586465.988300     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586465.988342     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json

# Cargar el archivo JSON
with open('tokenizer.json') as json_file:
    tokenizer_json = json_file.read()

# Reconstruir el tokenizer desde el archivo JSON
tokenizer = tokenizer_from_json(tokenizer_json)

In [5]:
def create_conv1d():
    model = models.Sequential()
    model.add(layers.Input(shape=(2172, 1)))
    model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    return model

In [6]:
def create_lstm():
    model=models.Sequential()
    # Primera capa LSTM con return_sequences=True
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.3)) # Añadir Dropout
    
    # Segunda capa LSTM con return_sequences=True
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.3))

    return model

In [7]:
def create_cnn_lstm_model(num_classes):
    conv1d = create_conv1d()
    
    video_input = layers.Input(shape=(None, 2172, 1)) 
    # masked_input = layers.Masking(mask_value=0.0)(video_input)

    
    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(conv1d)(video_input)
    
    lstm=create_lstm()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='linear')(lstm_out)
    
    model = models.Model(inputs=video_input, outputs=output)
    return model

In [8]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(references, hypotheses):
    smoothing = SmoothingFunction().method4
    return np.mean([sentence_bleu([ref], hyp, smoothing_function=smoothing) for ref, hyp in zip(references, hypotheses)])

In [10]:
num_classes=len(tokenizer.word_index) + 1
model = create_cnn_lstm_model(num_classes)

optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizador)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/best_model.keras',
    save_best_only=True,       # Guardar solo si es el mejor modelo hasta ahora
    monitor='val_loss',
    mode='min'  
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)

I0000 00:00:1728586470.095621     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586470.097565     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586470.097744     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586473.683146     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728586473.683215     518 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-10

In [11]:
num_classes

7664

In [12]:
ctc_loss=tf.keras.losses.CTC()

In [13]:
def decode(predictions,greedy):
    predicted_classes = tf.argmax(predictions, axis=-1)
    mask = tf.not_equal(predicted_classes, 0)
    seq_lengths = tf.reduce_sum(tf.cast(mask, tf.int32), axis=-1)
    
    predictions=tf.transpose(predictions,[1,0,2])

    if greedy:
        decoded, _ = tf.nn.ctc_greedy_decoder(inputs=predictions, sequence_length=seq_lengths)
    
    else:
        decoded, _ = tf.nn.ctc_beam_search_decoder(inputs=predictions, sequence_length=seq_lengths)
    decoded_dense = tf.sparse.to_dense(decoded[0], default_value=-1).numpy()
    
    # Extraer las secuencias decodificadas eliminando los -1 (valores de relleno)
    decoded_sequences = []
    for seq in decoded_dense:
        decoded_sequences.append([val for val in seq if val != -1])
    
    return decoded_sequences

In [14]:
!nvidia-smi

Thu Oct 10 18:54:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   47C    P5              22W /  35W |    503MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
train_dataset=tf.data.Dataset.load('./train').unbatch()
val_dataset=tf.data.Dataset.load('./val').unbatch()
test_dataset=tf.data.Dataset.load('./test').unbatch()

In [16]:
def length_fn(points, translations):
    # La longitud será la cantidad de keyframes (primera dimensión de los puntos)
    longitud=tf.shape(points)[0]
    return longitud


# limites = [300,500,700,1100]
    
# bucket_batch_sizes = [32, 32, 16, 8,8]

def bucketed_dataset(dataset):
    limites = [300,500,700,1100]
    
    bucket_batch_sizes = [4, 4,4, 4, 4]
    return dataset.bucket_by_sequence_length(
            element_length_func=length_fn,       
            bucket_boundaries=limites, # Límites de los buckets
            bucket_batch_sizes=bucket_batch_sizes,  # Tamaños de batch para cada bucket
            padded_shapes=(
                # [None,48,48],
                [None, 2172],  
                [None]           
            ),
            padding_values=(
                tf.constant(0, dtype=tf.float32), # Relleno con ceros para los puntos 
                tf.constant(0, dtype=tf.int32) # Relleno con ceros para las traducciones 
            )
    )

# Aplicar bucketing y padding
train_dataset= train_dataset.apply(bucketed_dataset)
val_dataset= val_dataset.apply(bucketed_dataset)
test_dataset= test_dataset.apply(bucketed_dataset)

In [17]:
!nvidia-smi

Thu Oct 10 18:54:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   47C    P5              21W /  35W |    503MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
from tqdm import tqdm
points_batch, translation_batch = next(iter(train_dataset))
points_batch.shape, translation_batch.shape

(TensorShape([4, 453, 2172]), TensorShape([4, 277]))

In [19]:
steps_per_epoch = train_dataset.cardinality().numpy()
validation_steps = val_dataset.cardinality().numpy()

In [20]:
# Entrenar el modelo usando un ciclo de entrenamiento personalizado
epochs = 100
model.compile(optimizer=optimizador, loss=tf.keras.losses.CTC(), metrics=[calculate_bleu])


# Iterar a través de los batches
for epoch in range(epochs):
  with tf.device('/GPU:0'):
    with tqdm(total=steps_per_epoch, desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    for epoch in tqdm(range(epochs), desc="Epoch"):

        for step, (points_batch, translation_batch) in enumerate(train_dataset):
            pbar.set_postfix({
                "Loss": epoch_loss_avg.result().numpy(),
                "Accuracy": epoch_accuracy.result().numpy()
            })
            
            with tf.GradientTape() as tape:
                predictions = model(points_batch, training=True)
                loss = tf.reduce_mean(ctc_loss(translation_batch, predictions))
                
            # Calcular y aplicar gradientes
            gradients = tape.gradient(loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            predictions_decoded=decode(predictions, greedy=True)
            epoch_accuracy.update_state(translation_batch, predictions_decoded)
            epoch_loss_avg.update_state(loss)
            
            
        
        # Métricas para la validación
        val_loss_avg = tf.keras.metrics.Mean()
        val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
        val_bleu_avg = tf.keras.metrics.Mean()
        with tqdm(total=validation_steps, desc=f'Epoch {epoch+1}/{epochs} (Validation)', unit='batch') as pbar_val:

            for step, (points_batch, translation_batch) in enumerate(val_dataset):
                
                pbar_val.set_postfix({
                    "Val Loss": val_loss_avg.result().numpy(),
                    "Val Accuracy": val_acc_metric.result().numpy(),
                    "Val BLEU": val_bleu_avg.result().numpy()
                })
                
                # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
                predictions = model(points_batch, training=False)

                val_loss=tf.reduce_mean(ctc_loss(translation_batch,predictions))
                val_loss_avg.update_state(val_loss)

                val_predictions = decode(predictions, greedy=False)
                val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in val_predictions]


                val_acc_metric.update_state(translation_batch,val_predictions)
                val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
                val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
                val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
                val_bleu_avg.update_state(val_bleu)
            
            

        checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg})

        if early_stopping.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg}):
            print("Early stopping triggered")
            break
        print(f"Epoch: {epoch}, Loss: {epoch_loss_avg}, Accuracy: {epoch_accuracy}")
        print(f'Epoch: {epoch}, val_loss: {val_loss_avg}, val_acc: {val_acc_metric}, val_bleu: {val_bleu_avg}')

    # Evaluar el modelo en el dataset de test
    #test_loss, test_acc = model.evaluate(test_dataset)
    #print(f'Test accuracy: {test_acc}')

Epoch 1/100: 0batch [00:00, ?batch/s]
Epoch:   0%|                                                                                                                                  | 0/100 [00:00<?, ?it/s]2024-10-10 18:54:44.913446: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:1728586487.489335     518 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728586487.530873     518 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728586487.538483     518 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728586487.548140     518 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728586487.560029     518 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728586487.573126     518 gpu_timer.cc:114] Skipping the delay kern